In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/skycheng17/Documents/GitHub/skycheng17.github.io/portfolio/6_Digital_Equity':
  os.chdir(r'/Users/skycheng17/Documents/GitHub/skycheng17.github.io/portfolio/6_Digital_Equity')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


/var/folders/yy/ysn331dd05s_vz0kmr85qzw40000gn/T/ipykernel_12851/998911783.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"/Users/skycheng17/mambaforge/envs/musa-550-fall-2023/lib/python3.10/importlib/_bootstrap.py": 1687560475.899881, "/Users/skycheng17/mambaforge/envs/musa-550-fall-2023/lib/python3.10/importlib/_bootstrap_external.py": 1687560475.905339, "/Users/skycheng17/mambaforge/envs/musa-550-fall-2023/lib/python3.10/codecs.py": 1687560473.470777, "/Users/skycheng17/mambaforge/envs/musa-550-fall-2023/lib/python3.10/encodings/aliases.py": 1687560474.989399, "/Users/skycheng17/mambaforge/envs/musa-550-fall-2023/lib/python3.10/encodings/__init__.py": 1687560474.985129, "/Users/skycheng17/mambaforge/envs/musa-550-fall-2023/lib/python3.10/encodings/utf_8.py": 1687560475.43757, "/Users/skycheng17/mambaforge/envs/musa-550-fall-2023/lib/python3.10/abc.py": 1687560473.346976, "/Users/skycheng17/mambaforge/envs/musa-550-fall-2023/lib/python3.10/io.py": 1687560473.627098, "/Users/skycheng17/mambaforge/envs/musa-550-fall-2023/lib/python3.10/stat.py": 1687560473.825673, "/Users/skycheng17/mambaforge/envs/musa-

In [2]:
#| code-fold: true

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import altair as alt
import geopandas as gpd

color_key = {"GSM": "#707390",
             "CDMA": "#93b7be",
             "UMTS": "#e0ca3c",
             "LTE": "#b59ab8",
             "NR": "#048a81"}

gdp_per_capita = pd.read_csv("./Data/gdp_per_capita.csv") #data from world bank
gdp_per_capita = gdp_per_capita[['Country Name', '2021 [YR2021]']]
gdp_per_capita = gdp_per_capita.rename(columns={
    'Country Name': 'country',
    '2021 [YR2021]': 'gdp_per_capita'
})
gdp_per_capita = gdp_per_capita.dropna(subset=['country'])
population = pd.read_csv("./data/population.csv") #data from world bank
population = population[['Country Name', '2021 [YR2021]']]
population = population.rename(columns={
    'Country Name': 'country',
    '2021 [YR2021]': 'population'
})
population = population.dropna(subset=['country'])
gpc_pop = population.merge(gdp_per_capita, how='left', on='country')

countries = gpd.read_file("https://datahub.io/core/geo-countries/r/countries.geojson")
countries = countries[['ADMIN', 'geometry']]
countries = countries.rename(columns={'ADMIN': 'country'})

gpc_pop_countries = countries.merge(gpc_pop, how='left', on='country')

countries_towers = pd.read_csv("./Data/international_cell_towers.csv")

merge = countries_towers.merge(gpc_pop_countries, how='left', on='country')
merge.rename(columns={'UMES': 'UMTS'}, inplace=True)

tower_types = ['UMTS', 'GSM', 'LTE', 'CDMA']
merge['dominant_tower_type'] = merge[tower_types].idxmax(axis=1)

merge['cell_towers'] = pd.to_numeric(merge['cell_towers'], errors='coerce')
merge['population'] = pd.to_numeric(merge['population'], errors='coerce')
merge['gdp_per_capita'] = pd.to_numeric(merge['gdp_per_capita'], errors='coerce')
merge['ctw_per_capita'] = merge['cell_towers'] / merge['population']

merge_df = pd.DataFrame(merge)
merge_df.drop(columns=['geometry'], inplace=True)
merge_df = merge_df.dropna()

brush = alt.selection_interval(encodings=['x'])
click = alt.selection_point(encodings=['color'])

points = alt.Chart(merge_df).mark_circle().encode(
    alt.X("gdp_per_capita:Q", scale=alt.Scale(zero=False, type="log"), title="Log GDP per Capita"),
    alt.Y("ctw_per_capita:Q", scale=alt.Scale(type="log"), title="Log Cell Towers per Capita"),
    size=alt.Size("population:Q", title="Population"),
    color=alt.condition(brush, alt.Color("dominant_tower_type:N", title="Dominant Tower Type", scale=alt.Scale(domain=list(color_key.keys()), range=list(color_key.values())),), alt.value('grey')),
    tooltip="country",
).properties(width=500, height=350).add_selection(brush).transform_filter(
    click
)

bars = alt.Chart(merge_df).mark_bar().encode(
    x='count()',
    y=alt.Y("dominant_tower_type:N", title="Dominant Tower Type"),
    color=alt.condition(brush, alt.Color("dominant_tower_type:N", title="Dominant Tower Type"), alt.value('lightgray')),
).transform_filter(brush).properties(width=550).add_params(
    click
)

# Combine charts
alt.vconcat(points, bars, title="World Cell Tower Coverage")

alt.VConcatChart(...)